# To Convert PyTorch Model to TensorFlow Lite

### Step - 1: Convert PyTorch Model to ONNX

In [1]:
import os

print(os.getcwd())


c:\Users\Akhil\Python_Projects\CNN-Multi-Class-Classifier


In [9]:
# pip install onnx


   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
    --------------------------------------- 0.3/14.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.5 MB 3.7 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/14.5 MB 3.0 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/14.5 MB 2.2 MB/s eta 0:00:06
   ----- ---------------------------------- 1.8/14.5 MB 1.9 MB/s eta 0:00:07
   ----- ---------------------------------- 2.1/14.5 MB 1.8 MB/s eta 0:00:07
   ------ --------------------------------- 2.4/14.5 MB 1.8 MB/s eta 0:00:07
   ------- -------------------------------- 2.6/14.5 MB 1.8 MB/s eta 0:00:07
   ------- -------------------------------- 2.9/14.5 MB 1.7 MB/s eta 0:00:07
   --------- ------------------------------ 3.4/14.5 MB 1.7 MB/s eta 0:00:07
   --------- ------------------------------ 3.4/14.5 MB 1.7 MB/s eta 0:00:07
   ---------- ----------------------------- 3.9/14.5 MB 1.6 MB/s eta 0:00:07
   ----------

In [2]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F

# Define the DeepCNN model architecture
class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # 3 channels (RGB)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # Adjust these dimensions to match your model
        self.fc2 = nn.Linear(128, 3)  # 3 classes: car, person, dog

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)  # Flatten the output
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 1: Instantiate the model
model = DeepCNN()

# Step 2: Load the saved model's state dictionary
model.load_state_dict(torch.load('deep_cnn_model.pth'))

# Step 3: Set the model to evaluation mode
model.eval()



# Step 4: Create a dummy input matching the input size of your model (3x224x224 for an RGB image)
dummy_input = torch.randn(1, 3, 224, 224)

# Step 5: Export the model to ONNX
torch.onnx.export(model, dummy_input, 'deep_cnn_model.onnx', opset_version=11)


C:\Users\Akhil\AppData\Local\Temp\ipykernel_8620\382827454.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('deep_cnn_model.pth'))


In [1]:
import onnx

# Load the ONNX model
model = onnx.load("deep_cnn_model.onnx")

# Get the graph from the model
graph = model.graph

# Print input nodes
print("Input Nodes:")
for input in graph.input:
    print(f"Name: {input.name}, Shape: {input.type.tensor_type.shape.dim}")

# Print output nodes
print("\nOutput Nodes:")
for output in graph.output:
    print(f"Name: {output.name}, Shape: {output.type.tensor_type.shape.dim}")


Input Nodes:
Name: input.1, Shape: [dim_value: 1
, dim_value: 3
, dim_value: 224
, dim_value: 224
]

Output Nodes:
Name: 24, Shape: [dim_value: 1
, dim_value: 3
]


In [2]:
import onnx

# Try loading the ONNX model to ensure it's not corrupted
try:
    model = onnx.load("deep_cnn_model.onnx")
    onnx.checker.check_model(model)
    print("ONNX model is valid!")
except Exception as e:
    print(f"Error loading ONNX model: {e}")


ONNX model is valid!


In [4]:
import torch
import torch.onnx
from model import DeepCNN  # Assuming your model class is in deep_cnn_model.py

# Load your model
model = DeepCNN()
model.load_state_dict(torch.load('deep_cnn_model.pth'))
model.eval()

# Dummy input for the ONNX export (assuming input images are 3x224x224 RGB)
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX
torch.onnx.export(model, dummy_input, 'model.onnx', opset_version=11)


C:\Users\Akhil\AppData\Local\Temp\ipykernel_14276\2053466231.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('deep_cnn_model.pth'))


### Alternative way to check input & output nodes is to upload your model file in Netron (https://netron.app/)

In [5]:
import onnx

# Load the ONNX model
model = onnx.load("model.onnx")

# Get the graph from the model
graph = model.graph

# Print input nodes
print("Input Nodes:")
for input in graph.input:
    print(f"Name: {input.name}, Shape: {input.type.tensor_type.shape.dim}")

# Print output nodes
print("\nOutput Nodes:")
for output in graph.output:
    print(f"Name: {output.name}, Shape: {output.type.tensor_type.shape.dim}")


Input Nodes:
Name: input.1, Shape: [dim_value: 1
, dim_value: 3
, dim_value: 224
, dim_value: 224
]

Output Nodes:
Name: 24, Shape: [dim_value: 1
, dim_value: 3
]


In [7]:
import tensorflow_addons as tfa
print(tfa.__version__)

ModuleNotFoundError: No module named 'tensorflow_addons'